In [2]:
'''Measure the latency matrix in parallal'''

'Measure the latency matrix in parallal'

In [3]:
'''Deploy cross-node communication measuring pods'''

from kubernetes import client, config

def check_pods_existence(namespace):
    core_v1 = client.CoreV1Api()
    try:
        pods = core_v1.list_namespaced_pod(namespace=namespace)
        return len(pods.items) > 0 # return True
    except client.rest.ApiException as e:
        print(f"Exception when calling CoreV1Api->list_namespaced_pod: {e}")
        return False

def deploy_latency_measurement_daemonset_if_needed():
    namespace = 'measure-nodes'
    ds_name = 'latency-measurement-ds'
    
    # Assuming config and apps_v1 have been defined as before
    config.load_kube_config()
    apps_v1 = client.AppsV1Api()

    # Check if pods exist in the namespace
    pods_exist = check_pods_existence(namespace)

    # If no pods exist, deploy the DaemonSet
    if not pods_exist:
        # Define the body of the DaemonSet
        ds_body = client.V1DaemonSet(
            api_version="apps/v1",
            kind="DaemonSet",
            metadata=client.V1ObjectMeta(name=ds_name),
            spec=client.V1DaemonSetSpec(
                selector=client.V1LabelSelector(
                    match_labels={"app": "latency-measurement"}
                ),
                template=client.V1PodTemplateSpec(
                    metadata=client.V1ObjectMeta(labels={"app": "latency-measurement"}),
                    spec=client.V1PodSpec(
                        containers=[client.V1Container(
                            name="latency-container",
                            image="curlimages/curl",
                            security_context=client.V1SecurityContext(
                                capabilities=client.V1Capabilities(
                                    add=["NET_RAW"]  # Required for ping
                                )
                            )
                        )],
                        restart_policy="Always"
                    )
                )
            )
        )

        # Deploy the DaemonSet
        try:
            apps_v1.create_namespaced_daemon_set(namespace=namespace, body=ds_body)
            print(f"Deployed DaemonSet {ds_name} in namespace {namespace}")
        except client.rest.ApiException as e:
            print(f"Exception when calling AppsV1Api->create_namespaced_daemon_set: {e}")
    else:
        print("Pods already exist in the namespace. Skipping DaemonSet deployment.")

# Call the function to deploy the DaemonSet if needed
deploy_latency_measurement_daemonset_if_needed()



Pods already exist in the namespace. Skipping DaemonSet deployment.


In [ ]:

'''As there are network jitter sometimes, means soemtimes sudden rise for a signle measurement:
To sovle this measuring issue:
(1) a easy and simple way to do is just the choose the lowest measured vaule to represent the latency 
between the measured node pair.
(2) another way, that can do is to meaured more vaules (like meaure 20 times), and use the 80% percentile as
the latency.

'''

from kubernetes import client, config, stream
import concurrent.futures

# Handle individual latency measurements between source and target pods.
def measure_latency_from_source_to_target(v1, namespace, source_pod, target_pod):
    source_pod_name = source_pod.metadata.name
    source_pod_node_name = source_pod.spec.node_name
    target_pod_ip = target_pod.status.pod_ip
    target_pod_name = target_pod.metadata.name
    target_pod_node_name = target_pod.spec.node_name
    result = (source_pod_node_name, target_pod_node_name, None)

    if source_pod_name != target_pod_name:
        exec_command = ['curl', '-o', '/dev/null', '-s', '-w', '%{time_total}', f'http://{target_pod_ip}']
        latencies = []
        for _ in range(5):
            try:
                resp = stream.stream(v1.connect_get_namespaced_pod_exec,
                                     source_pod_name,
                                     namespace,
                                     command=exec_command,
                                     stderr=True,
                                     stdin=False,
                                     stdout=True,
                                     tty=False)
                latency = float(resp) * 1000  # Convert the measured seconds into milliseconds
                latencies.append(latency)
            except Exception as e:
                print(f"Error executing command in pod {source_pod_name}: {e}")
        
        if latencies:
            min_latency = min(latencies) #choose the lowest measured vaule to represent the latency
            result = (source_pod_node_name, target_pod_node_name, min_latency)
            print(f"Latency from source node {source_pod_node_name} to target node {target_pod_node_name}: {min_latency} milliseconds")
        else:
            result = (source_pod_node_name, target_pod_node_name, "Error")

    return result

def measure_http_latency(namespace='measure-nodes'):
    v1 = client.CoreV1Api()
    pods = v1.list_namespaced_pod(namespace, label_selector="app=latency-measurement").items
    latency_results = {}
    
    # Run the latency measurement tasks concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for source_pod in pods:
            for target_pod in pods:
                futures.append(executor.submit(measure_latency_from_source_to_target, v1, namespace, source_pod, target_pod))
        
        # Aggregate the completed results into the latency_results dictionary
        for future in concurrent.futures.as_completed(futures):
            source_pod_node_name, target_pod_node_name, latency = future.result()
            if source_pod_node_name not in latency_results:
                latency_results[source_pod_node_name] = {}
            latency_results[source_pod_node_name][target_pod_node_name] = latency

    return latency_results

# from kubernetes import client, config
# import concurrent.futures

# def get_uncordoned_nodes():
#     """Return a set of node names that are schedulable (not cordoned)."""
#     v1 = client.CoreV1Api()
#     nodes = v1.list_node().items
#     return {n.metadata.name for n in nodes if not getattr(n.spec, "unschedulable", False)}

# def measure_http_latency(namespace='measure-nodes'):
#     config.load_kube_config()
#     v1 = client.CoreV1Api()

#     # 1️⃣ Get all pods in the target namespace
#     pods = v1.list_namespaced_pod(namespace, label_selector="app=latency-measurement").items

#     # 2️⃣ Get schedulable (uncordoned) nodes
#     uncordoned_nodes = get_uncordoned_nodes()

#     # 3️⃣ Keep only pods that are on uncordoned nodes
#     active_pods = [p for p in pods if p.spec.node_name in uncordoned_nodes]

#     print(f"[INFO] Found {len(active_pods)} latency pods on uncordoned nodes: {[p.spec.node_name for p in active_pods]}")

#     latency_results = {}

#     # 4️⃣ Run latency measurement tasks concurrently
#     with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#         futures = []
#         for source_pod in active_pods:
#             for target_pod in active_pods:
#                 futures.append(
#                     executor.submit(measure_latency_from_source_to_target,
#                                     v1, namespace, source_pod, target_pod)
#                 )

#         # 5️⃣ Collect results
#         for future in concurrent.futures.as_completed(futures):
#             try:
#                 source_node, target_node, latency = future.result()
#                 latency_results.setdefault(source_node, {})[target_node] = latency
#             except Exception as e:
#                 print(f"[WARN] Measurement failed: {e}")

#     return latency_results


# Call the function to measure latency after deploying the DaemonSet
namespace = 'measure-nodes'
# After the DaemonSet is ready, and the related pods are ready
latency_results = measure_http_latency(namespace=namespace)
print(latency_results)


Latency from source node k8s-worker-6 to target node k8s-worker-3: 3.5300000000000002 milliseconds
Latency from source node k8s-worker-6 to target node k8s-worker-7: 10.57 milliseconds
Latency from source node k8s-worker-6 to target node k8s-worker-5: 7.495 milliseconds
Latency from source node k8s-worker-6 to target node k8s-worker-9: 18.540999999999997 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-6: 3.49 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-5: 10.534999999999998 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-9: 14.561 milliseconds
Latency from source node k8s-worker-9 to target node k8s-worker-4: 13.597 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-7: 7.538 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-2: 15.655 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-8: 3.758 milliseconds
Latency from 

In [ ]:
import pandas as pd

# Convert the nested dictionary into a pandas DataFrame
df = pd.DataFrame(latency_results)

# Transpose the DataFrame to align the source and destination workers as per convention
df = df.T

# Fill diagonal with 0's for self-latency (optional, if desired for clarity)
for worker in df.columns:
    df.at[worker, worker] = 0

df

In [ ]:
''' Normalize node-node communication delay matrix'''


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Convert the  dictionary into a pandas DataFrame and transpose it
df = pd.DataFrame(latency_results).T

# Mapping old names to new names
worker_node = {
    'k8s-worker-1': 'node-1', 'k8s-worker-2': 'node-2', 'k8s-worker-3': 'node-3',
    'k8s-worker-4': 'node-4', 'k8s-worker-5': 'node-5', 'k8s-worker-6': 'node-6',
    'k8s-worker-7': 'node-7', 'k8s-worker-8': 'node-8', 'k8s-worker-9': 'node-9'
}

# Rename the columns and index according to the dictionary
df.rename(columns=worker_node, index=worker_node, inplace=True)

# Fill diagonal with 0's for self-latency (optional, if desired for clarity)
for worker in df.columns:
    df.at[worker, worker] = 0

# Normalize the DataFrame
max_latency = df.max().max()  # Find the maximum value in the DataFrame
normalized_df = df / max_latency  # Divide every element by the maximum value

# Plot the normalized matrix
plt.figure(figsize=(10, 8))
sns.heatmap(normalized_df, annot=True,  cmap= "Reds",fmt=".2f", annot_kws={'size': 12},xticklabels=True, yticklabels=True, linewidths=.5)
# plt.title('Normalized Latency Matrix')
plt.xlabel('Destination Nodes', fontsize=15)
plt.ylabel('Source Nodes', fontsize=15)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# k8s-worker --- node
woker_node = {'k8s-worker-1': 'node1', 'k8s-worker-2': 'node2', 'k8s-worker-3': 'node3',
              'k8s-worker-4': 'node4', 'k8s-worker-5': 'node5', 'k8s-worker-6': 'node6',
              'k8s-worker-7': 'node7', 'k8s-worker-8': 'node8', 'k8s-worker-9': 'node9'}



df = pd.DataFrame(latency_results).T
for worker in df.columns:
    df.at[worker, worker] = 0

# Calculate the correlation matrix
corr_matrix = df.corr()

# Invert the correlation matrix
# inverted_corr_matrix =  np.abs(corr_matrix)

# Plot the inverted correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Inverted Correlation Matrix')
plt.show()



In [ ]:
# Convert the nested dictionary into a pandas DataFrame and transpose it
df = pd.DataFrame(latency_results).T

# Fill diagonal with 0's for self-latency
for worker in df.columns:
    df.at[worker, worker] = 0

# Calculate the correlation matrix
corr_matrix = df.corr()

# Dictionary to map old worker names to new names
worker_node = {
    'k8s-worker-1': 'node-1', 'k8s-worker-2': 'node-2', 'k8s-worker-3': 'node-3',
    'k8s-worker-4': 'node-4', 'k8s-worker-5': 'node-5', 'k8s-worker-6': 'node-6',
    'k8s-worker-7': 'node-7', 'k8s-worker-8': 'node-8', 'k8s-worker-9': 'node-9'
}

# Rename the columns and index of the correlation matrix for visualization
renamed_corr_matrix = corr_matrix.rename(columns=worker_node, index=worker_node)

# Take the absolute value of the correlation matrix to invert it
inverted_renamed_corr_matrix = np.abs(renamed_corr_matrix)

# Plot the inverted correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(inverted_renamed_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", xticklabels=worker_node.values(), yticklabels=worker_node.values())
plt.title('Inverted Correlation Matrix with Node Labels')
plt.xlabel('Node')
plt.ylabel('Node')
plt.show()